Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. По образу практики, попробуйте создать искусственный датасет с лишними столбцами. Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [126]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

У меня будет фейковый датасет тиражей школьных учебников. Это, наверное, единственные книги, у которых до сих пор большие тиражи. При этом учебники печатают часто, допечатки могут быть чуть ли не каждый месяц, то есть у большого издательства может набраться датасет из тысячи тиражей довольно быстро.

Таргетная характеристика - количество жалоб от пользователей. Она зависит от следующих показателей:
1. Наличие ошибок в тексте учебника, которые не заметили при подготовке файла для типографии. Не может быть в явном виде представлена в датасете: если в тексте учебника есть ошибки, значит, их пока никто не заметил, а если их заметили, в следующем тираже их уже не будет. Поэтому условимся считать, что при подготовке каждого следующего тиража учебник дополнительно проверяют, то есть вероятность, что там есть ошибки, уменьшается. Пусть вероятность найти ошибку в первом тираже учебника составляет 5 %. Тогда, чтобы сохранить линейность функции, мы будем вычислять, сколько книг составляют 5 % от тиража, и делить это число на номер издания. Получившееся число войдет в общее количество рекламаций.
2. Процент типографских ошибок. При отпечатке учебника могут возникнуть всякие неприятные вещи вроде перепутанных страниц, пропущенных страниц, смещения печатного поля и т.д. Не все их заметят покупатели, но они тоже могут стать причиной жалоб. Пусть на типографские ошибки жалуется один из десяти пострадавших покупателей. Мы будем вычислять, сколько книг поступило в продажу с типографским браком, и делить это число на 10.

Свободный член сделала отрицательным, потому что не все покупатели пожалуются на плохое качество учебника. Добавила условие, при котором если результат получается отрицательным, функция возвращает 0. Интересно посмотреть, как это отразится на качестве модели.
Получившийся результат округляется, потому что количество жалоб не может быть дробным числом.

Описание признаков:
* edition - номер издания (1-31)
* edition_size - размер тиража (1000-100 000)
* printing_defects - процент типографского брака (до 25 %)
* book_size - размер учебника, стр. (50-350), на таргетную переменную не влияет
* cource - учебный предмет (категориальная переменная), на таргетную переменную не влияет
* school_class - уровень образования (категориальная переменная), на таргетную переменную не влияет
* complaints - количество жалоб от пользователей (таргетная переменная)

In [127]:
n_samples = 1000

edition = np.random.choice(30, n_samples) + 1
edition_size = (np.random.choice(100, n_samples) + 1) * 1000
printing_defects = np.random.choice(25, n_samples)
book_size = np.random.choice(300, n_samples) + 50
cource = np.random.choice(['math', 'biology', 'physics', 'reading', 'defense against the dark arts'], n_samples)
school_class = np.random.choice(['primary school', 'middle school', 'high school'], n_samples)

complaints = list(map(lambda x: round(x) if x > 0 else 0,
                      edition_size / 1000 * printing_defects + edition_size / 20 / edition - 214))
complaints = np.array(complaints)

data = pd.DataFrame({'edition': edition, 'edition_size': edition_size, 'printing_defects': printing_defects,
                     'book_size': book_size, 'cource': cource, 'school_class': school_class, 'complaints': complaints})
data.head()

,edition,edition_size,printing_defects,book_size,cource,school_class,complaints
0,10,21000,4,333,defense against the dark arts,primary school,0
1,10,73000,6,228,reading,middle school,589
2,6,74000,11,312,math,primary school,1217
3,6,93000,21,304,math,high school,2514
4,15,77000,5,348,biology,high school,428


In [128]:
# Линейная регрессия по всем переменным
X = pd.get_dummies(data.loc[:, ['edition', 'edition_size', 'printing_defects', 'book_size', 'cource', 'school_class']])
y = data['complaints']

X.head()

,edition,edition_size,printing_defects,book_size,cource_biology,cource_defense against the dark arts,cource_math,cource_physics,cource_reading,school_class_high school,school_class_middle school,school_class_primary school
0,10,21000,4,333,0,1,0,0,0,0,0,1
1,10,73000,6,228,0,0,0,0,1,0,1,0
2,6,74000,11,312,0,0,1,0,0,0,0,1
3,6,93000,21,304,0,0,1,0,0,1,0,0
4,15,77000,5,348,1,0,0,0,0,1,0,0


In [129]:
reg = LinearRegression().fit(X, y)
predicted = reg.predict(X)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
print(f'Error: {mean_absolute_error(predicted, y)}')

Weights: [-3.59860332e+01  1.77338263e-02  5.27952287e+01  1.73624095e-01
  5.29338129e+01 -1.90199068e+01 -3.77007771e+00 -4.43505175e+01
  1.42066891e+01 -2.04221857e+01  3.19380955e+01 -1.15159098e+01]
Bias: -262.0426082819406
Error: 317.3595944435515


In [130]:
# Линейная регрессия без категориальных переменных и размера книги
X = data[['edition', 'edition_size', 'printing_defects']]
y = data['complaints']

X.head()

,edition,edition_size,printing_defects
0,10,21000,4
1,10,73000,6
2,6,74000,11
3,6,93000,21
4,15,77000,5


In [131]:
reg = LinearRegression().fit(X, y)
predicted = reg.predict(X)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
print(f'Error: {mean_absolute_error(predicted, y)}')

Weights: [-3.60028014e+01  1.78042878e-02  5.26538462e+01]
Bias: -226.3145463365338
Error: 316.0646428455621


In [132]:
# Вместо printing_defects добавляю количество книг с дефектом в тираже
data['abs_defects'] = data['edition_size'] / 100 * data['printing_defects']
data.head()

,edition,edition_size,printing_defects,book_size,cource,school_class,complaints,abs_defects
0,10,21000,4,333,defense against the dark arts,primary school,0,840.0
1,10,73000,6,228,reading,middle school,589,4380.0
2,6,74000,11,312,math,primary school,1217,8140.0
3,6,93000,21,304,math,high school,2514,19530.0
4,15,77000,5,348,biology,high school,428,3850.0


In [133]:
X = data[['edition', 'edition_size', 'abs_defects']]
y = data['complaints']

X.head()

,edition,edition_size,abs_defects
0,10,21000,840.0
1,10,73000,4380.0
2,6,74000,8140.0
3,6,93000,19530.0
4,15,77000,3850.0


In [134]:
reg = LinearRegression().fit(X, y)
predicted = reg.predict(X)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
print(f'Error: {mean_absolute_error(predicted, y)}')

Weights: [-3.46202309e+01  4.98345325e-03  1.07681656e-01]
Bias: 375.50367137053036
Error: 261.4132664282452


In [135]:
#Объединяю edition и edition_size в edition_size / edition.
# Об этом довольно сложно догадаться в реальной жизни, но раз уж у меня условный пример, то почему бы и нет.
# Логика примерно такая: чем больше номер тиража, тем лучше вычитана книжка, тем меньше поводов жаловаться.
# Чем больше размер тиража, тем больше книжек с ошибками попадет к читателям, тем больше поводов жаловаться.
data['text_errors'] = data['edition_size'] / data['edition']
data.head()

,edition,edition_size,printing_defects,book_size,cource,school_class,complaints,abs_defects,text_errors
0,10,21000,4,333,defense against the dark arts,primary school,0,840.0,2100.000000
1,10,73000,6,228,reading,middle school,589,4380.0,7300.000000
2,6,74000,11,312,math,primary school,1217,8140.0,12333.333333
3,6,93000,21,304,math,high school,2514,19530.0,15500.000000
4,15,77000,5,348,biology,high school,428,3850.0,5133.333333


In [136]:
X = data[['text_errors', 'abs_defects']]
y = data['complaints']

X.head()

,text_errors,abs_defects
0,2100.000000,840.0
1,7300.000000,4380.0
2,12333.333333,8140.0
3,15500.000000,19530.0
4,5133.333333,3850.0


In [137]:
reg = LinearRegression().fit(X, y)
predicted = reg.predict(X)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
print(f'Error: {mean_absolute_error(predicted, y)}')

Weights: [0.0496346  0.09703402]
Bias: -176.88608692677406
Error: 28.138259159365127


Ошибка очень сильно уменьшилась, но все равно довольно большая. Вероятно, это связано с тем, что функция не возвращает отрицательные значения и работает с округлениями. В данном случае я могу предположить, что количество рекламаций не может быть отрицательным, и заменить в предсказании все отрицательные числа нулями. Это уменьшит абсолютную погрешность

In [138]:
predicted_2 = [x if x > 0 else 0 for x in predicted]
print(f'Error: {mean_absolute_error(predicted_2, y)}')

Error: 16.574257177567233


Теперь изменю формулу таргетной переменной так, чтобы она лучше соответствовала линейной регрессии.

In [139]:
# Сначала проверяю датасет, где в таргетной переменной возможны отрицательные значения
n_samples = 1000

edition = np.random.choice(30, n_samples) + 1
edition_size = (np.random.choice(100, n_samples) + 1) * 1000
printing_defects = np.random.choice(25, n_samples)
abs_defects = edition_size / 100 * printing_defects
text_errors = edition_size / edition

complaints = list(map(int,
                      edition_size / 1000 * printing_defects + edition_size / 20 / edition - 214))
complaints = np.array(complaints)

data = pd.DataFrame({'edition': edition, 'edition_size': edition_size, 'printing_defects': printing_defects,
                     'abs_defects': abs_defects, 'text_errors': text_errors, 'complaints': complaints})
data.head()

,edition,edition_size,printing_defects,abs_defects,text_errors,complaints
0,15,35000,23,8050.0,2333.333333,707
1,4,54000,14,7560.0,13500.000000,1217
2,22,37000,5,1850.0,1681.818182,55
3,23,92000,24,22080.0,4000.000000,2194
4,19,38000,21,7980.0,2000.000000,684


In [140]:
X = data[['abs_defects', 'text_errors']]
y = data['complaints']

In [141]:
reg = LinearRegression().fit(X, y)
predicted = reg.predict(X)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
print(f'Error: {mean_absolute_error(predicted, y)}')

Weights: [0.09996836 0.05000369]
Bias: -214.0379336291902
Error: 0.30897543158974955


In [142]:
# Теперь проверяю датасет, где возможны дробные значения
n_samples = 1000

edition = np.random.choice(30, n_samples) + 1
edition_size = (np.random.choice(100, n_samples) + 1) * 1000
printing_defects = np.random.choice(25, n_samples)
abs_defects = edition_size / 100 * printing_defects
text_errors = edition_size / edition

complaints = list(map(lambda x: x if x > 0 else 0,
                      edition_size / 1000 * printing_defects + edition_size / 20 / edition - 214))
complaints = np.array(complaints)

data = pd.DataFrame({'edition': edition, 'edition_size': edition_size, 'printing_defects': printing_defects,
                     'abs_defects': abs_defects, 'text_errors': text_errors, 'complaints': complaints})
data.head()

,edition,edition_size,printing_defects,abs_defects,text_errors,complaints
0,10,43000,11,4730.0,4300.000000,474.000000
1,2,17000,11,1870.0,8500.000000,398.000000
2,8,29000,7,2030.0,3625.000000,170.250000
3,23,22000,1,220.0,956.521739,0.000000
4,3,86000,24,20640.0,28666.666667,3283.333333


In [143]:
X = data[['abs_defects', 'text_errors']]
y = data['complaints']

In [144]:
reg = LinearRegression().fit(X, y)
predicted = reg.predict(X)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
print(f'Error: {mean_absolute_error(predicted, y)}')

Weights: [0.09684605 0.04951461]
Bias: -171.996604117704
Error: 30.48404522762582


Таким образом, округление не сильно влияет на работу линейной регрессии в отличие от принудительного избавления от отрицательных чисел